In [11]:
import pandas as pd
import re

# Load data
data = pd.read_csv("Corona_NLP_train.csv")

# Text preprocessing function
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()                  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces and strip
    return text

# Apply preprocessing
data['clean_text'] = data['text'].apply(preprocess)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encode labels
label_encoder = LabelEncoder()
data['encoded_labels'] = label_encoder.fit_transform(data['category'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['encoded_labels'], test_size=0.2, random_state=42)

from gensim.models import FastText
import numpy as np

# Train FastText model
sentences = [text.split() for text in X_train]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=2, sg=1)

# Function to get FastText embeddings
def get_fasttext_vectors(text, model, vector_size):
    tokens = text.split()
    vector = np.zeros(vector_size)
    valid_tokens = 0
    for token in tokens:
        if token in model.wv:
            vector += model.wv[token]
            valid_tokens += 1
    return vector / valid_tokens if valid_tokens > 0 else vector

# Convert training and test sets to FastText embeddings
X_train_fasttext = np.array([get_fasttext_vectors(text, fasttext_model, 100) for text in X_train])
X_test_fasttext = np.array([get_fasttext_vectors(text, fasttext_model, 100) for text in X_test])

from gensim.models import FastText
import numpy as np

# Train FastText model
sentences = [text.split() for text in X_train]
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=2, sg=1)

# Function to get FastText embeddings
def get_fasttext_vectors(text, model, vector_size):
    tokens = text.split()
    vector = np.zeros(vector_size)
    valid_tokens = 0
    for token in tokens:
        if token in model.wv:
            vector += model.wv[token]
            valid_tokens += 1
    return vector / valid_tokens if valid_tokens > 0 else vector

# Convert training and test sets to FastText embeddings
X_train_fasttext = np.array([get_fasttext_vectors(text, fasttext_model, 100) for text in X_train])
X_test_fasttext = np.array([get_fasttext_vectors(text, fasttext_model, 100) for text in X_test])

from gensim.downloader import load
import numpy as np

# Load pre-trained GloVe model
glove_model = load("glove-wiki-gigaword-100")

# Function to get GloVe embeddings
def get_glove_vectors(text, model, vector_size):
    tokens = text.split()
    vector = np.zeros(vector_size)
    valid_tokens = 0
    for token in tokens:
        if token in model:
            vector += model[token]
            valid_tokens += 1
    return vector / valid_tokens if valid_tokens > 0 else vector

# Convert training and test sets to GloVe embeddings
X_train_glove = np.array([get_glove_vectors(text, glove_model, 100) for text in X_train])
X_test_glove = np.array([get_glove_vectors(text, glove_model, 100) for text in X_test])

from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Initialize BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_vectors(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Convert training and test sets to BERT embeddings
X_train_bert = np.array([get_bert_vectors(text, bert_tokenizer, bert_model) for text in X_train])
X_test_bert = np.array([get_bert_vectors(text, bert_tokenizer, bert_model) for text in X_test])


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 7485: invalid continuation byte